In [5]:
import requests

url = "https://data.ampmetropole.fr/api/explore/v2.0/catalog/datasets/disponibilites-des-places-de-parkings/records"

params = {
    "limit": 100,
    "where": "commune='AIX EN PROVENCE'"
}

response = requests.get(url, params=params)
data = response.json()

print(f"Parkings trouvés: {data['total_count']}\n")

for parking in data['records']:
    fields = parking['record']['fields']
    nom = fields['nom']
    dispo = fields['voitureplacesdisponibles']
    capacite = fields['voitureplacescapacite']
    temps_reel = fields['tempsreel']
    
    print(f"{nom}: {dispo}/{capacite} places | Temps réel: {temps_reel}")

Parkings trouvés: 15

Hauts de Brunet: 366.0/366.0 places | Temps réel: False
Bellegarde: 340.0/340.0 places | Temps réel: False
Cardeurs: 125.0/125.0 places | Temps réel: False
Mignet: 800.0/800.0 places | Temps réel: False
Plan d'Aillane: 300.0/300.0 places | Temps réel: False
Carnot: 675.0/675.0 places | Temps réel: False
Krypton: 865.0/865.0 places | Temps réel: False
Malacrida: 242.0/242.0 places | Temps réel: False
Lieutenant-Colonel Jeanpierre: 600.0/600.0 places | Temps réel: False
Route des Alpes: 124.0/124.0 places | Temps réel: False
Arena: 210.0/210.0 places | Temps réel: False
Rotonde: 1500.0/1500.0 places | Temps réel: False
Mejanes: 800.0/800.0 places | Temps réel: False
Pasteur: 650.0/650.0 places | Temps réel: False
Signoret: 350.0/350.0 places | Temps réel: False


In [37]:
import requests

url = "https://data.ampmetropole.fr/api/explore/v2.0/catalog/datasets/disponibilites-des-places-de-parkings/records"

params = {
    "limit": 100,
    "where": "commune='AIX EN PROVENCE' AND tempsreel='True'"
}

response = requests.get(url, params=params)
data = response.json()

print(f"Parkings avec temps réel: {data['total_count']}\n")

for parking in data['records']:
    fields = parking['record']['fields']
    nom = fields['nom']
    dispo = fields['voitureplacesdisponibles']
    capacite = fields['voitureplacescapacite']
    
    print(f"{nom}: {dispo}/{capacite} places")

Parkings avec temps réel: 0



In [9]:
import requests
import re
import time
import pandas as pd
from scrapy import Selector

url = "https://mamp.parkings-semepa.fr/"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

r = requests.get(url, headers=headers)
response = Selector(text=r.text)
r

<Response [200]>

In [36]:
import requests
import re
import time

parkings = {
    'Bellegarde': ('https://mamp.parkings-semepa.fr/', 213),
    'Cardeurs': ('https://mamp.parkings-semepa.fr/', 219),
    'Carnot': ('https://mamp.parkings-semepa.fr/', 211),
    'Méjanes': ('https://mamp.parkings-semepa.fr/', 150),
    'Mignet': ('https://mamp.parkings-semepa.fr/', 209),
    'Pasteur': ('https://mamp.parkings-semepa.fr/', 215),
    'Rambot': ('https://parkings-semepa.fr/', 221),
    'Rotonde': ('https://parkings-semepa.fr/', 206),
    'Signoret': ('https://mamp.parkings-semepa.fr/', 217)
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

print(f"🔄 Scraping de {len(parkings)} parkings...\n")

for nom, (base_url, page_id) in parkings.items():
    try:
        response = requests.get(base_url, params={"page_id": page_id}, headers=headers, timeout=5)
        
        # Chercher <p class="nbPlaces"><span...>nombre</span>
        match_nombre = re.search(r'<p class="nbPlaces"><span[^>]*>(\d+)</span>', response.text)
        match_texte = re.search(r'<p class="nbPlaces"><span[^>]*>([^<]+)</span>', response.text)
        
        if match_nombre:
            places_libres = match_nombre.group(1)
            print(f"✓ {nom:15} | {places_libres} places libres")
        elif match_texte:
            statut = match_texte.group(1)
            print(f"⚠️  {nom:15} | {statut}")
        else:
            print(f"⚠️  {nom:15} | Pas de données temps réel")
        
    except Exception as e:
        print(f"❌ {nom:15} | Erreur: {e}")
    
    time.sleep(1)

print("\n✓ Test terminé")

🔄 Scraping de 9 parkings...

✓ Bellegarde      | 9 places libres
✓ Cardeurs        | 33 places libres
✓ Carnot          | 193 places libres
⚠️  Méjanes         | Fermeture temporaire
✓ Mignet          | 217 places libres
✓ Pasteur         | 28 places libres
✓ Rambot          | 78 places libres
✓ Rotonde         | 445 places libres
⚠️  Signoret        | Pas de données temps réel

✓ Test terminé
